# * VINSIGHT : Metric Insight

## Parameter

In [213]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [214]:
''' Parameter Input '''

# v_metric_cd = 'B0R000100' #Total Revenue
# v_metric_cd = 'DB0R000100' #Total Revenue : DTAC
# v_metric_cd = 'TB0R000100' #Total Revenue : TRUE
# v_metric_cd = 'B0R000100GEO' #Total Revenue (Geo)
# v_metric_cd = 'DB0R000100GEO' #Total Revenue (Geo) : DTAC
# v_metric_cd = 'TB0R000100GEO' #Total Revenue (Geo) : TRUE

# v_metric_cd = 'B0R000101' #Mobile Revenue
# v_metric_cd = 'DB0R000101' #Mobile Revenue : DTAC
# v_metric_cd = 'TB0R000101' #Mobile Revenue : TMH
# v_metric_cd = 'B0R000101GEO' #Mobile Revenue (Geo)
# v_metric_cd = 'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
# v_metric_cd = 'TB0R000101GEO' #Mobile Revenue (Geo) : TMH

# v_metric_cd = 'B1R000100' #Prepaid Revenue
# v_metric_cd = 'DB1R000100' #Prepaid Revenue : DTAC
# v_metric_cd = 'TB1R000100' #Prepaid Revenue : TMH

# v_metric_cd = 'B2R000100' #Postpaid Revenue
# v_metric_cd = 'B2R010100' #Postpaid Revenue B2C
# v_metric_cd = 'B2R020100' #Postpaid Revenue B2B
# v_metric_cd = 'DB2R000100' #Postpaid Revenue : DTAC
# v_metric_cd = 'DB2R010100' #Postpaid Revenue B2C : DTAC
# v_metric_cd = 'DB2R020100' #Postpaid Revenue B2B : DTAC
# v_metric_cd = 'TB2R000100' #Postpaid Revenue : TMH
v_metric_cd = 'TB2R010100' #Postpaid Revenue B2C : TMH
# v_metric_cd = 'TB2R020100' #Postpaid Revenue B2B : TMH

# v_metric_cd = 'TB3R000100' #TOL Revenue

# v_metric_cd = 'TB4R000100' #TVS Revenue

# v_metric_cd = 'TB2S010600' #Postpaid Reported SubBase B2C : TMH

# v_metric_cd = 'B0R00010001CS' #Total Inflow M1
# v_metric_cd = 'DB0R00010001CS' #Total Inflow M1 : DTAC
# v_metric_cd = 'TB0R00010001CS' #Total Inflow M1 : TRUE

# v_metric_cd = 'B1R000900CS' #Prepaid Inflow M1
# v_metric_cd = 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
# v_metric_cd = 'TB1R000900CS' #Prepaid Inflow M1 : TMH

# v_metric_cd = 'B2R010500CS' #Postpaid Inflow M1 B2C
# v_metric_cd = 'B2R020500CS' #Postpaid Inflow M1 B2B
# v_metric_cd = 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
# v_metric_cd = 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
# v_metric_cd = 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
# v_metric_cd = 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

# v_metric_cd = 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
# v_metric_cd = 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
# v_metric_cd = 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
# v_metric_cd = 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

# v_metric_cd = 'TB4R001004CS' #TVS CMDU Inflow M1
# v_metric_cd = 'TB4R001700CS' #TVS Now Inflow M1

v_period_grp = 'Monthly' #Yearly, Quarterly, Monthly, Weekly, Daily

v_area_cd = 'GX1'

# v_start_date = 20240101

print(f'\nParameter input...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_period_grp: {v_period_grp}')
print(f'   -> v_area_cd: {v_area_cd}')


Parameter input...

   -> v_metric_cd: TB2R010100
   -> v_period_grp: Monthly
   -> v_area_cd: GX1


## Import Transaction

In [215]:
''' Execute transaction '''

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_WK, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        , TARGET_SNAP, TARGET_AGG_WK, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = '{v_metric_cd}'
    --AND TM_KEY_DAY >= v_start_date
    --ORDER BY TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\n   -> DataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Data as of 2025-06-05, 12:20:04

   -> DataFrame: 111510 rows, 24 columns


In [216]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [217]:
# ''' Sample Data '''

# chk_src_df.tail()

## Overview by Period

In [218]:
''' Yearly Summary '''

yearly_df = chk_src_df.copy()
yearly_df['P_TARGET'] = np.where(yearly_df['AREA_CD']=='P', yearly_df['TARGET_SNAP'], 0)
yearly_df['C'] = np.where(yearly_df['AREA_CD']=='C', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['P'] = np.where(yearly_df['AREA_CD']=='P', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['G'] = np.where(yearly_df['AREA_TYPE']=='G', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['H'] = np.where(yearly_df['AREA_TYPE']=='H', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['HH'] = np.where(yearly_df['AREA_TYPE']=='HH', yearly_df['ACTUAL_SNAP'], 0)

yearly_df = yearly_df.groupby(['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
yearly_df['P_YoY'] = yearly_df['P'] - yearly_df['P'].shift()
yearly_df = yearly_df.fillna(0).sort_values(by=['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

yearly_df_display = yearly_df.copy()
yearly_df_display['ACTUAL_AS_OF'] = yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = yearly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    yearly_df_display[col] = yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
yearly_df_display

,TM_KEY_YR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH,P_YoY
0,2023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20231231,"33,245,012,566",0,"32,854,818,250","32,543,728,912","32,737,963,624",0,0
1,2024,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20241231,"20,009,779,223","19,718,511,948","19,718,511,948","19,686,024,184","19,686,023,030","19,686,023,030","-13,136,306,302"
2,2025,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,"17,263,745,807","16,705,501,532","16,705,501,532","16,698,674,044","16,698,659,902","16,698,659,902","-3,013,010,416"


In [219]:
''' Quarterly Summary '''

quarterly_df = chk_src_df.copy()
quarterly_df['P_TARGET'] = np.where(quarterly_df['AREA_CD']=='P', quarterly_df['TARGET_SNAP'], 0)
quarterly_df['C'] = np.where(quarterly_df['AREA_CD']=='C', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['P'] = np.where(quarterly_df['AREA_CD']=='P', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['G'] = np.where(quarterly_df['AREA_TYPE']=='G', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['H'] = np.where(quarterly_df['AREA_TYPE']=='H', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['HH'] = np.where(quarterly_df['AREA_TYPE']=='HH', quarterly_df['ACTUAL_SNAP'], 0)

quarterly_df = quarterly_df.groupby(['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
quarterly_df['P_QoQ'] = quarterly_df['P'] - quarterly_df['P'].shift()
quarterly_df = quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

quarterly_df_display = quarterly_df.copy()
quarterly_df_display['ACTUAL_AS_OF'] = quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = quarterly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    quarterly_df_display[col] = quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))

quarterly_df_display

,TM_KEY_QTR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH,P_QoQ
0,20231,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230331,"3,316,616,982",0,"3,233,961,809","3,207,262,907","3,226,149,679",0,0
1,20232,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230630,"9,990,498,738",0,"9,786,703,099","9,704,815,161","9,762,505,383",0,"6,552,741,290"
2,20233,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230930,"9,954,064,200",0,"9,885,358,090","9,798,875,131","9,857,447,809",0,"98,654,991"
3,20234,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20231231,"9,983,832,647",0,"9,948,795,252","9,832,775,713","9,891,860,754",0,"63,437,162"
4,20241,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0,0,"-9,948,795,252"
5,20242,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0,0,0
6,20243,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20240930,"9,987,507,408","9,834,028,205","9,834,028,205","9,818,595,523","9,818,594,946","9,818,594,946","9,834,028,205"
7,20244,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20241231,"10,022,271,815","9,884,483,743","9,884,483,743","9,867,428,661","9,867,428,084","9,867,428,084","50,455,538"
8,20251,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250331,"10,046,030,199","9,989,547,936","9,989,547,936","9,982,743,196","9,982,733,231","9,982,733,231","105,064,193"
9,20252,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,"7,217,715,608","6,715,953,597","6,715,953,597","6,715,930,848","6,715,926,671","6,715,926,671","-3,273,594,339"


In [220]:
''' Monthly Summary '''

v_tm_key_yr = curr_yr
# v_tm_key_yr = 2025

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_YR']==v_tm_key_yr].copy()
monthly_df['P_TARGET'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['TARGET_SNAP'], 0)
monthly_df['C'] = np.where(monthly_df['AREA_CD']=='C', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['P'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['G'] = np.where(monthly_df['AREA_TYPE']=='G', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['H'] = np.where(monthly_df['AREA_TYPE']=='H', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['HH'] = np.where(monthly_df['AREA_TYPE']=='HH', monthly_df['ACTUAL_SNAP'], 0)

monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df['P_MoM'] = monthly_df['P'] - monthly_df['P'].shift()
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH,P_MoM
0,202501,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250131,"3,331,779,353","3,336,770,490","3,336,770,490","3,329,988,784","3,329,984,467","3,329,984,467",0
1,202502,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250228,"3,347,051,673","3,333,288,223","3,333,288,223","3,333,276,337","3,333,273,586","3,333,273,586","-3,482,267"
2,202503,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250331,"3,367,199,173","3,319,489,222","3,319,489,222","3,319,478,075","3,319,475,179","3,319,475,179","-13,799,001"
3,202504,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250430,"3,358,275,973","3,371,023,854","3,371,023,854","3,371,013,229","3,371,011,525","3,371,011,525","51,534,631"
4,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250531,"3,373,190,486","3,344,929,743","3,344,929,743","3,344,917,619","3,344,915,146","3,344,915,146","-26,094,111"
5,202506,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,"486,249,148",0,0,0,0,0,"-3,344,929,743"


In [221]:
''' Weekly Summary '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
weekly_df['P_TARGET'] = np.where(weekly_df['AREA_CD']=='P', weekly_df['TARGET_SNAP'], 0)
weekly_df['C'] = np.where(weekly_df['AREA_CD']=='C', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['P'] = np.where(weekly_df['AREA_CD']=='P', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['G'] = np.where(weekly_df['AREA_TYPE']=='G', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['H'] = np.where(weekly_df['AREA_TYPE']=='H', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['HH'] = np.where(weekly_df['AREA_TYPE']=='HH', weekly_df['ACTUAL_SNAP'], 0)

weekly_df = weekly_df.groupby(['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
weekly_df['P_WoW'] = weekly_df['P'] - weekly_df['P'].shift()
weekly_df = weekly_df.fillna(0).sort_values(by=['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

weekly_df_display = weekly_df.copy()
weekly_df_display['ACTUAL_AS_OF'] = weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = weekly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    weekly_df_display[col] = weekly_df_display[col].apply(lambda x: format(x, ',.0f'))

weekly_df_display

,TM_KEY_WK,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH,P_WoW
0,2025018,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250502,"486,249,148","479,978,969","479,978,969","479,978,050","479,977,403","479,977,403",0
1,2025019,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250510,"723,600,099","732,140,891","732,140,891","732,137,519","732,137,519","732,137,519","252,161,922"
2,2025020,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250516,"756,360,805","747,019,496","747,019,496","747,018,457","747,016,731","747,016,731","14,878,605"
3,2025021,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250525,"1,047,613,151","1,043,588,358","1,043,588,358","1,043,584,812","1,043,584,712","1,043,584,712","296,568,862"
4,2025022,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250531,"359,367,283","342,202,028","342,202,028","342,198,780","342,198,780","342,198,780","-701,386,331"
5,2025023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,"486,249,148",0,0,0,0,0,"-342,202,028"


In [222]:
''' Daily Summary '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

# daily_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_tm_key_mth].copy()
daily_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
daily_df['P_TARGET'] = np.where(daily_df['AREA_CD']=='P', daily_df['TARGET_SNAP'], 0)
daily_df['C'] = np.where(daily_df['AREA_CD']=='C', daily_df['ACTUAL_SNAP'], 0)
daily_df['P'] = np.where(daily_df['AREA_CD']=='P', daily_df['ACTUAL_SNAP'], 0)
daily_df['G'] = np.where(daily_df['AREA_TYPE']=='G', daily_df['ACTUAL_SNAP'], 0)
daily_df['H'] = np.where(daily_df['AREA_TYPE']=='H', daily_df['ACTUAL_SNAP'], 0)
daily_df['HH'] = np.where(daily_df['AREA_TYPE']=='HH', daily_df['ACTUAL_SNAP'], 0)

daily_df = daily_df.groupby(['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
daily_df = daily_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# daily_df['P_DoD'] = daily_df['P'] - daily_df['P'].shift()

daily_df_display = daily_df.copy()
mod_col_list = daily_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    daily_df_display[col] = daily_df_display[col].apply(lambda x: format(x, ',.0f'))

daily_df_display

,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P_TARGET,C,P,G,H,HH
0,20250501,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
1,20250502,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,"486,249,148","479,978,969","479,978,969","479,978,050","479,977,403","479,977,403"
2,20250503,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
3,20250504,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
4,20250505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
5,20250506,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,"267,643,342","287,818,841","287,818,841","287,818,084","287,818,084","287,818,084"
6,20250507,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
7,20250508,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
8,20250509,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,0,0,0,0,0,0
9,20250510,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,"455,956,757","444,322,050","444,322,050","444,319,435","444,319,435","444,319,435"


## Area Type Summary

In [223]:
def agg_period(v_period_grp):

    # Get : Parameter
    period_grp = v_period_grp
    result = 0

    if period_grp == 'Yearly': result = curr_yr
    elif period_grp == 'Quarterly': result = curr_qtr
    elif period_grp == 'Monthly': result = curr_mth
    elif period_grp == 'Weekly': result = curr_wk
    else: result = curr_day
 
    return result


# Check Result
print(f'\nagg_period: {agg_period(v_period_grp)}')


agg_period: 202506


In [224]:
''' Auto Prep Aggregate by Period '''

if v_period_grp == 'Yearly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Quarterly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_QTR', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Monthly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Weekly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_WK', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Daily':
    raw_period_df = chk_src_df.groupby(['TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
else:
    print(f'\n*** Aggregate Period Input ? ***')


''' Add Columns '''

raw_period_df['GAP'] = raw_period_df['ACTUAL_SNAP'] - raw_period_df['TARGET_SNAP']
# raw_period_df['%ACH'] = raw_period_df['ACTUAL_SNAP'] / raw_period_df['TARGET_SNAP'] * 100
raw_period_df['%ACH'] = raw_period_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)

# raw_period_df['%PORTION'] = raw_period_df['ACTUAL_SNAP'] / raw_period_df.groupby([raw_period_df.columns[0], 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# raw_period_df

raw_period_df_display = raw_period_df.copy()
raw_period_df_display = raw_period_df_display.fillna(0).sort_values(by=[raw_period_df_display.columns[0], 'METRIC_CD', 'AREA_TYPE', 'AREA_CD']).reset_index(drop=True)
raw_period_df_display['ACTUAL_AS_OF'] = raw_period_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = raw_period_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    raw_period_df_display[col] = raw_period_df_display[col].apply(lambda x: format(x, ',.0f'))
raw_period_df_display['%ACH'] = raw_period_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
# raw_period_df_display['%PORTION'] = raw_period_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
# raw_period_df_display

In [225]:
''' P : Nationwide Summary '''

p_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
p_summary_df = p_summary_df.loc[p_summary_df['AREA_TYPE']=='P']
p_summary_df = p_summary_df.fillna(0).sort_values(by=[p_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
p_summary_df['%PORTION'] = p_summary_df['ACTUAL_SNAP'] / p_summary_df.groupby([p_summary_df.columns[0], 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# p_summary_df = p_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

p_summary_df_display = p_summary_df.copy()
p_summary_df_display['ACTUAL_AS_OF'] = p_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = p_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    p_summary_df_display[col] = p_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
p_summary_df_display['%ACH'] = p_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
p_summary_df_display['%PORTION'] = p_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
p_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,P,P,Nationwide,0,"486,249,148","-486,249,148",0.00,nan


In [226]:
''' G : 8 Region Summary '''

g_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
g_summary_df = g_summary_df.loc[g_summary_df['AREA_TYPE']=='G']
g_summary_df['%PORTION'] = g_summary_df['ACTUAL_SNAP'] / g_summary_df.groupby([g_summary_df.columns[0], 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# g_summary_df = g_summary_df.fillna(0).sort_values(by=[g_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
g_summary_df = g_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

g_summary_df_display = g_summary_df.copy()
g_summary_df_display['ACTUAL_AS_OF'] = g_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = g_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    g_summary_df_display[col] = g_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
g_summary_df_display['%ACH'] = g_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
g_summary_df_display['%PORTION'] = g_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
g_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX1,BMA : West,0,"68,611,925","-68,611,925",0.00,0.00
1,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX2,BMA : East,0,"82,925,280","-82,925,280",0.00,0.00
2,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX3,East,0,"71,368,844","-71,368,844",0.00,0.00
3,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX4,North,0,"59,187,313","-59,187,313",0.00,0.00
4,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX5,Northeast 1,0,"53,574,109","-53,574,109",0.00,0.00
5,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX6,Northeast 2,0,"49,267,134","-49,267,134",0.00,0.00
6,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX7,"Central, West",0,"60,632,206","-60,632,206",0.00,0.00
7,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-04 23:49:33,20250602,G,GX8,South,0,"40,682,337","-40,682,337",0.00,0.00


In [227]:
''' H : 64 Province Summary '''

h_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
h_summary_df = h_summary_df.loc[h_summary_df['AREA_TYPE']=='H']
h_summary_df['%PORTION'] = h_summary_df['ACTUAL_SNAP'] / h_summary_df.groupby([h_summary_df.columns[0], 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# h_summary_df = h_summary_df.fillna(0).sort_values(by=[h_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
h_summary_df = h_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

h_summary_df_display = h_summary_df.copy()
h_summary_df_display['ACTUAL_AS_OF'] = h_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = h_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    h_summary_df_display[col] = h_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
h_summary_df_display['%ACH'] = h_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
h_summary_df_display['%PORTION'] = h_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
h_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,003,"SMP : Bang Bo, Bang Sao Thong, Bang Phli",0,"9,020,336","-9,020,336",0.00,0.00
1,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,004,"SMP : Mueang Samut Prakan, Phra Pradaeng, Phra...",0,"12,754,387","-12,754,387",0.00,0.00
2,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,38X,"Bueng Kan, Nong Khai",0,"4,291,987","-4,291,987",0.00,0.00
3,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,39X,"Udon Thani, Nong Bua Lam Phu",0,"8,827,153","-8,827,153",0.00,0.00
4,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,40Z,Khon Kaen,0,"9,785,054","-9,785,054",0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,24X,"Chachoengsao, Nakhon Nayok",0,"7,770,733","-7,770,733",0.00,0.00
60,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,26X,"Sa Kaeo, Prachin Buri",0,"5,695,593","-5,695,593",0.00,0.00
61,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,30X,Nakhon Ratchasima,0,"15,511,905","-15,511,905",0.00,0.00
62,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 01:26:29,20250602,H,31Z,Buri Ram,0,"7,665,001","-7,665,001",0.00,0.00


In [228]:
''' HH : 96 Cluster Summary '''

hh_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
hh_summary_df = hh_summary_df.loc[hh_summary_df['AREA_TYPE']=='HH']
hh_summary_df['%PORTION'] = hh_summary_df['ACTUAL_SNAP'] / hh_summary_df.groupby([hh_summary_df.columns[0], 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# hh_summary_df = hh_summary_df.fillna(0).sort_values(by=[hh_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
hh_summary_df = hh_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

hh_summary_df_display = hh_summary_df.copy()
hh_summary_df_display['ACTUAL_AS_OF'] = hh_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = hh_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    hh_summary_df_display[col] = hh_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
hh_summary_df_display['%ACH'] = hh_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
hh_summary_df_display['%PORTION'] = hh_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
hh_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,902033,Ang Thong,0,"1,515,244","-1,515,244",0.00,0.00
1,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,902034,Chai Nat,0,"1,878,006","-1,878,006",0.00,0.00
2,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,908005,"BKK : Taling Chan, Phasi Charoen, Thawi Wattha...",0,"10,731,751","-10,731,751",0.00,0.00
3,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,908004,"BKK : Thon Buri, Khlong San, Rat Burana, Thung...",0,"6,300,267","-6,300,267",0.00,0.00
4,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,908003,"BKK : Bangkok Yai, Bangkok Noi, Bang Phlat",0,"4,751,925","-4,751,925",0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,904064,Chiang Rai,0,"6,797,585","-6,797,585",0.00,0.00
92,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,904063,Chiang Mai 2,0,"5,814,223","-5,814,223",0.00,0.00
93,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,904062,Chiang Mai 1,0,"7,083,118","-7,083,118",0.00,0.00
94,202506,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,20250602,HH,903061,Trat,0,"1,517,334","-1,517,334",0.00,0.00


## Area Insight

In [229]:
print(f'\nv_area_cd: {v_area_cd}')


v_area_cd: GX1


In [230]:
''' Yearly by Area '''

yearly_area_df = chk_src_df.loc[chk_src_df['AREA_CD']==v_area_cd].copy()
# yearly_area_df = yearly_area_df.groupby(['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'})
yearly_area_df = yearly_area_df.groupby(['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'TARGET_SNAP':'sum'})
yearly_area_df = yearly_area_df.fillna(0).sort_values(by=['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
yearly_area_df['GAP'] = yearly_area_df['ACTUAL_SNAP'] - yearly_area_df['TARGET_SNAP']
yearly_area_df['%ACH'] = yearly_area_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)
# yearly_area_df['%PORTION'] = yearly_area_df['ACTUAL_SNAP'] / yearly_area_df.groupby(['TM_KEY_YR', 'METRIC_CD', 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
yearly_area_df['ACTUAL_AS_OF'] = yearly_area_df['ACTUAL_AS_OF'].astype(int)

yearly_area_df_display = yearly_area_df.copy()
mod_col_list = yearly_area_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    yearly_area_df_display[col] = yearly_area_df_display[col].apply(lambda x: format(x, ',.0f'))
yearly_area_df_display['%ACH'] = yearly_area_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
yearly_area_df_display

,TM_KEY_YR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,AREA_CD,AREA_NAME,PPN_TM,ACTUAL_AS_OF,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,2023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,Regional Management 1 (BMA : West),2024-04-23 12:22:16,20231231,"4,668,300,702","4,825,895,065","-157,594,364",96.73
1,2024,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20241231,"2,817,061,267","2,848,581,102","-31,519,835",98.89
2,2025,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,"2,637,112,003","2,435,991,594","201,120,409",108.26


In [231]:
''' Quarterly by Area '''

quarterly_area_df = chk_src_df.loc[chk_src_df['AREA_CD']==v_area_cd].copy()
quarterly_area_df = quarterly_area_df.groupby(['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'TARGET_SNAP':'sum'})
quarterly_area_df = quarterly_area_df.fillna(0).sort_values(by=['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
quarterly_area_df['GAP'] = quarterly_area_df['ACTUAL_SNAP'] - quarterly_area_df['TARGET_SNAP']
quarterly_area_df['%ACH'] = quarterly_area_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)
quarterly_area_df['ACTUAL_AS_OF'] = quarterly_area_df['ACTUAL_AS_OF'].astype(int)

quarterly_area_df_display = quarterly_area_df.copy()
mod_col_list = quarterly_area_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    quarterly_area_df_display[col] = quarterly_area_df_display[col].apply(lambda x: format(x, ',.0f'))
quarterly_area_df_display['%ACH'] = quarterly_area_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
quarterly_area_df_display

,TM_KEY_QTR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,AREA_CD,AREA_NAME,PPN_TM,ACTUAL_AS_OF,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,20231,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,Regional Management 1 (BMA : West),2024-04-23 12:22:16,20230331,"461,762,158","479,764,814","-18,002,656",96.25
1,20232,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,Regional Management 1 (BMA : West),2024-04-23 12:22:16,20230630,"1,394,535,550","1,445,174,344","-50,638,794",96.50
2,20233,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,Regional Management 1 (BMA : West),2024-04-23 12:22:16,20230930,"1,403,404,628","1,448,321,669","-44,917,042",96.90
3,20234,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,Regional Management 1 (BMA : West),2024-04-23 12:22:16,20231231,"1,408,598,366","1,452,634,238","-44,035,872",96.97
4,20241,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,0,0,0,0,0.00
5,20242,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,0,0,0,0,0.00
6,20243,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20240930,"1,402,130,332","1,426,062,332","-23,932,000",98.32
7,20244,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20241231,"1,414,930,935","1,422,518,771","-7,587,835",99.47
8,20251,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250331,"1,579,855,710","1,417,539,704","162,316,006",111.45
9,20252,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,"1,057,256,293","1,018,451,890","38,804,403",103.81


In [232]:
''' Monthly by Area '''

monthly_area_df = chk_src_df.loc[chk_src_df['AREA_CD']==v_area_cd].copy()
monthly_area_df = monthly_area_df.loc[monthly_area_df['TM_KEY_YR']==curr_yr]
monthly_area_df = monthly_area_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'TARGET_SNAP':'sum'})
monthly_area_df = monthly_area_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_area_df['GAP'] = monthly_area_df['ACTUAL_SNAP'] - monthly_area_df['TARGET_SNAP']
monthly_area_df['%ACH'] = monthly_area_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)
monthly_area_df['ACTUAL_AS_OF'] = monthly_area_df['ACTUAL_AS_OF'].astype(int)

monthly_area_df_display = monthly_area_df.copy()
mod_col_list = monthly_area_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    monthly_area_df_display[col] = monthly_area_df_display[col].apply(lambda x: format(x, ',.0f'))
monthly_area_df_display['%ACH'] = monthly_area_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
monthly_area_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,AREA_CD,AREA_NAME,PPN_TM,ACTUAL_AS_OF,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,202501,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250131,"526,943,439","470,128,939","56,814,501",112.08
1,202502,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250228,"527,305,836","472,283,931","55,021,905",111.65
2,202503,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250331,"525,606,435","475,126,834","50,479,601",110.62
3,202504,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250430,"530,412,416","473,867,730","56,544,686",111.93
4,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250531,"526,843,877","475,972,235","50,871,642",110.69
5,202506,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,0,"68,611,925","-68,611,925",0.00


In [233]:
''' Weekly by Area '''

weekly_area_df = chk_src_df.loc[chk_src_df['AREA_CD']==v_area_cd].copy()
weekly_area_df = weekly_area_df.loc[weekly_area_df['TM_KEY_MTH']==curr_mth]
weekly_area_df = weekly_area_df.groupby(['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'TARGET_SNAP':'sum'})
weekly_area_df = weekly_area_df.fillna(0).sort_values(by=['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
weekly_area_df['GAP'] = weekly_area_df['ACTUAL_SNAP'] - weekly_area_df['TARGET_SNAP']
weekly_area_df['%ACH'] = weekly_area_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)
weekly_area_df['ACTUAL_AS_OF'] = weekly_area_df['ACTUAL_AS_OF'].astype(int)

weekly_area_df_display = weekly_area_df.copy()
mod_col_list = weekly_area_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    weekly_area_df_display[col] = weekly_area_df_display[col].apply(lambda x: format(x, ',.0f'))
weekly_area_df_display['%ACH'] = weekly_area_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
weekly_area_df_display

,TM_KEY_WK,PRODUCT_GRP,METRIC_CD,METRIC_NAME,AREA_CD,AREA_NAME,PPN_TM,ACTUAL_AS_OF,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,2025022,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250531,0,0,0,0.00
1,2025023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,0,"68,611,925","-68,611,925",0.00


In [234]:
''' Daily by Area '''

daily_area_df = chk_src_df.loc[chk_src_df['AREA_CD']==v_area_cd].copy()
daily_area_df = daily_area_df.loc[daily_area_df['TM_KEY_MTH']==curr_mth]
daily_area_df = daily_area_df.groupby(['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'TARGET_SNAP':'sum'})
daily_area_df = daily_area_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
daily_area_df['GAP'] = daily_area_df['ACTUAL_SNAP'] - daily_area_df['TARGET_SNAP']
daily_area_df['%ACH'] = daily_area_df.apply(lambda rows: 0 if rows['TARGET_SNAP']==0 else rows['ACTUAL_SNAP']/rows['TARGET_SNAP']*100, axis=1)
daily_area_df['ACTUAL_AS_OF'] = daily_area_df['ACTUAL_AS_OF'].astype(int)

daily_area_df_display = daily_area_df.copy()
mod_col_list = daily_area_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    daily_area_df_display[col] = daily_area_df_display[col].apply(lambda x: format(x, ',.0f'))
daily_area_df_display['%ACH'] = daily_area_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
daily_area_df_display

,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,AREA_CD,AREA_NAME,PPN_TM,ACTUAL_AS_OF,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,20250601,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250531,0,0,0,0.00
1,20250602,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,0,"68,611,925","-68,611,925",0.00
2,20250603,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,GX1,BMA : West,2025-06-04 23:49:33,20250602,0,0,0,0.00
